In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Use tensorflow datasets to get the data
import tensorflow_datasets as tfds

(train_data,test_data), ds_info = tfds.load(name='sentiment140',split = ['train','test'],
                                            shuffle_files = False,with_info = True,
                                            as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/sentiment140/1.0.0.incomplete4KQFFQ/sentiment140-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/sentiment140/1.0.0.incomplete4KQFFQ/sentiment140-test.tfrecord


Dataset sentiment140 downloaded and prepared to /root/tensorflow_datasets/sentiment140/1.0.0. Subsequent calls will reuse this data.


In [33]:
df = tfds.as_dataframe(train_data, ds_info)

In [34]:
df.head()

,polarity,text
0,4,"b""i'm 10x cooler than all of you! """
1,0,b'O.kk? Thats weird I cant stop following peop...
2,4,b'what a beautiful day not to got to my first ...
3,4,"b"".@HildyGottlieb &amp; I was just saying to M..."
4,0,b'kinda sad and confused why do guys do this?'


In [46]:
df_test = tfds.as_dataframe(test_data, ds_info)

In [35]:
df['polarity'].value_counts()

4    800000
0    800000
Name: polarity, dtype: int64

In [66]:
df_test['polarity'].value_counts()

4    182
0    177
Name: polarity, dtype: int64

In [57]:
df_test.head(10)

,text
polarity,
0,b'looks like summize has gone down. too many t...
4,b'SOOO DISSAPOiNTED THEY SENT DANNY GOKEY HOME...
4,b'I seriously underestimated Malcolm Gladwell....
4,b'[#MLUC09] Customer Innovation Award Winner: ...
4,b'@shannyoday I will take you on a date to see...
0,"b'arhh, It\'s weka bug. = ="" and I spent almos..."
0,b'Ahh...got rid of stupid time warner today &a...
4,b'@ work til 6pm... lets go lakers!!!'
2,b'Your Normal Weight (and How to Get There) ? ...


In [58]:
df_test.drop(2,axis=0,inplace=True)
df_test.head(10)

,text
polarity,
0,b'looks like summize has gone down. too many t...
4,b'SOOO DISSAPOiNTED THEY SENT DANNY GOKEY HOME...
4,b'I seriously underestimated Malcolm Gladwell....
4,b'[#MLUC09] Customer Innovation Award Winner: ...
4,b'@shannyoday I will take you on a date to see...
0,"b'arhh, It\'s weka bug. = ="" and I spent almos..."
0,b'Ahh...got rid of stupid time warner today &a...
4,b'@ work til 6pm... lets go lakers!!!'
4,b'top ten most watched on Viral-Video Chart. ...


In [65]:
df_test = df_test.reset_index()

In [77]:
train_text = df['text'].tolist()
test_text = df_test['text'].tolist()

In [79]:
test_text[:10]

[b'looks like summize has gone down. too many tweets from WWDC perhaps?',
 b'SOOO DISSAPOiNTED THEY SENT DANNY GOKEY HOME... YOU STiLL ROCK ...DANNY ... MY HOMETOWN HERO !! YEAH MiLROCKEE!!',
 b'I seriously underestimated Malcolm Gladwell.  I want to meet this dude.',
 b'[#MLUC09] Customer Innovation Award Winner: Booz Allen Hamilton -- http://ping.fm/c2hPP',
 b'@shannyoday I will take you on a date to see night at the museum 2 whenever you want...it looks soooooo good',
 b'arhh, It\'s weka bug. = =" and I spent almost two hours to find that out. crappy me',
 b'Ahh...got rid of stupid time warner today &amp; now taking a nap while the roomies cook for me. Pretty good end for a monday :)',
 b'@ work til 6pm... lets go lakers!!!',
 b'top ten most watched on Viral-Video Chart.  Love the nike #mostvaluablepuppets campaign from Wieden &amp; Kennedy http://bit.ly/nR1n9',
 b'Recovering from surgery..wishing @julesrenner was here :(']

In [68]:
#one hot encoding
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

train_target = one_hot_encoder.fit_transform(df['polarity'].to_numpy().reshape(-1,1))
test_target = one_hot_encoder.transform(df_test['polarity'].to_numpy().reshape(-1,1))

In [69]:
train_target[:10]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [80]:
#Label Encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

train_target1 = label_encoder.fit_transform(df['polarity'].to_numpy())
test_target1 = label_encoder.transform(df_test['polarity'].to_numpy())

In [81]:
train_target1[:10]

array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0])

In [82]:
test_target1[:10]

array([0, 1, 1, 1, 1, 0, 0, 1, 1, 0])

In [4]:
class_names = ds_info.features
class_names

FeaturesDict({
    'date': Text(shape=(), dtype=tf.string),
    'polarity': tf.int32,
    'query': Text(shape=(), dtype=tf.string),
    'text': Text(shape=(), dtype=tf.string),
    'user': Text(shape=(), dtype=tf.string),
})

# Model_0 : Tensorflow Hub

In [106]:
round(sum([len(i.split()) for i in train_text])/len(train_text))
sent_length = [len(i.split()) for i in train_text]
output_len = int(np.percentile(sent_length,95))
output_len

25

In [108]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_vocab_size=50000
max_length = 25

text_vectorizer = TextVectorization(max_tokens=max_vocab_size,
                                    output_sequence_length=max_length,
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    pad_to_max_tokens=True)

#Fit text_vectorizer to training data
text_vectorizer.adapt(train_text)

In [109]:
from tensorflow.keras import layers

#converts each int token into a 128 dim vector randomly initialized

embedding = layers.Embedding(input_dim = max_vocab_size,
                             output_dim = 128,
                             mask_zero=True,
                             input_length=max_length,
                             name='token_embedding')

In [110]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,dropout=.5,return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64,dropout=.5))(x)
outputs = layers.Dense(1,activation='sigmoid')(x)

model_0 = tf.keras.Model(inputs,outputs)
  
model_0.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics = ['accuracy'])
  

In [111]:
#Creating dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_text,train_target1)) #Using one hot target
test_dataset = tf.data.Dataset.from_tensor_slices((test_text,test_target1))


train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [112]:
history_0  = model_0.fit(train_dataset,epochs=3,
                         steps_per_epoch = int(0.2*len(train_dataset)),
                         validation_data = test_dataset,
                         validation_steps = int(0.2*len(test_dataset))
                        )

Epoch 1/3
10000/10000 [==============================] - 791s 78ms/step - loss: 0.4477 - accuracy: 0.7897 - val_loss: 0.4103 - val_accuracy: 0.7969
Epoch 2/3
10000/10000 [==============================] - 772s 77ms/step - loss: 0.4100 - accuracy: 0.8119 - val_loss: 0.4059 - val_accuracy: 0.8125
Epoch 3/3
10000/10000 [==============================] - 785s 79ms/step - loss: 0.3990 - accuracy: 0.8174 - val_loss: 0.4189 - val_accuracy: 0.7969


In [113]:
model_0.evaluate(test_dataset)

12/12 [==============================] - 0s 9ms/step - loss: 0.3835 - accuracy: 0.8217


[0.38348156213760376, 0.8217270374298096]

In [114]:
model_0_pred_probs = model_0.predict(test_dataset)

In [115]:
model_0_pred = tf.squeeze(
    tf.round(model_0_pred_probs), axis=1)
model_0_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)>

In [116]:
from helper_function import calculate_results
calculate_results(test_target1,model_0_pred)

{'accuracy': 82.17270194986072,
 'f1': 0.8213580449202273,
 'precision': 0.82350550145379,
 'recall': 0.8217270194986073}

In [119]:
model_0.save('sentiment_model')

INFO:tensorflow:Assets written to: model.tf/assets


INFO:tensorflow:Assets written to: model.tf/assets
